In [1]:
import transformers
from transformers import AutoTokenizer
import torch
import os
print(f"number of GPUs: torch.cuda.device_count()")
print(torch.__version__)

number of GPUs: torch.cuda.device_count()
2.6.0+rocm6.1


In [2]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" 

if  torch.cuda.is_available():
    device = "cuda"
else:
    raise ValueError("No se reconoció GPU.")

pipeline = transformers.pipeline(
	"text-generation", 
	model=model_id,
	model_kwargs={"torch_dtype": torch.bfloat16},
	device=device
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda


In [ ]:
# Directorios de entrada y salida
input_dir = "txt/"
output_dir = "out/"
os.makedirs(output_dir, exist_ok=True)

# Instrucciones para el modelo
prompt = [
    {"role": "system", 
     "content": 
        """ 
        Quiero que identifiques entidades nombradas que requieren ser anonimizadas en el informe clínico que copio entre comillas al final de esta instrucción. Quiero que me des el resultado en formato .xml in-line, donde las entidades sean identificadas por etiquetas en el mismo texto. Quiero que etiquetes con los criterios MEDDOCAN. A continuación, te muestro un ejemplo que contiene:
        - El texto original del informe en formato plano (.txt)
        - La representación estructurada del mismo en XML con etiquetas semánticas detalladas y posiciones de texto (atributos start, end, text, TYPE, etc.).
        Tu tarea será generar un XML con las mismas reglas de estructura y etiquetado a partir de cada texto clínico. Instrucciones:
        - Conserva el formato exacto del XML del ejemplo.
        - Cada etiqueta tiene que tener el tipo de entidad (`TYPE`) del inventario de MEDDOCAN. Los tipos de entidad que puedes usar son los siguientes: 
        NOMBRE_SUJETO_ASISTENCIA
        EDAD_SUJETO_ASISTENCIA
        SEXO_SUJETO_ASISTENCIA
        FAMILIARES_SUJETO_ASISTENCIA
        NOMBRE_PERSONAL_SANITARIO
        FECHAS
        PROFESION
        HOSPITAL
        CENTRO_SALUD
        INSTITUCION
        CALLE
        TERRITORIO
        PAIS
        NUMERO_TELEFONO
        NUMERO_FAX
        CORREO_ELECTRONICO
        ID_SUJETO_ASISTENCIA
        ID_CONTACTO_ASISTENCIAL
        ID_ASEGURAMIENTO
        ID_TITULACION_PERSONAL_SANITARIO
        ID_EMPLEO_PERSONAL_SANITARIO
        IDENTIF_VEHICULOS_NRSERIE_PLACAS
        IDENTIF_DISPOSITIVOS_NRSERIE
        DIREC_PROT_INTERNET
        URL_WEB
        IDENTIF_BIOMETRICOS
        OTRO_NUMERO_IDENTIF
        OTROS_SUJETO_ASISTENCIA
          - y un campo de comentario (`comment`) vacío
        Cuando te dé un nuevo texto, responde solo con el XML, sin explicaciones adicionales.
        
        Ejemplo - Informe en formato .txt: 
        Datos del paciente.
        Nombre:  Ernesto.
        Apellidos: Rivera Bueno.
        NHC: 368503.
        NASS: 26 63514095.
        Domicilio:  Calle Miguel Benitez 90.
        Localidad/ Provincia: Madrid.
        CP: 28016.
        Datos asistenciales.
        Fecha de nacimiento: 03/03/1946.
        País: España.
        Edad: 70 años Sexo: H.
        Fecha de Ingreso: 12/12/2016.
        Médico:  Ignacio Navarro Cuéllar NºCol: 28 28 70973.
        Informe clínico del paciente: Paciente de 70 años de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
        Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
        En la exploración física presenta un buen estado general, con abdomen y genitales normales; tacto rectal compatible con adenoma de próstata grado I/IV.
        En la analítica de orina destaca la existencia de 4 hematíes/ campo y 0-5 leucocitos/campo; resto de sedimento normal.
        Hemograma normal; en la bioquímica destaca una glucemia de 169 mg/dl y triglicéridos de 456 mg/dl; función hepática y renal normal. PSA de 1.16 ng/ml.
        Las citologías de orina son repetidamente sospechosas de malignidad.
        En la placa simple de abdomen se valoran cambios degenerativos en columna lumbar y calcificaciones vasculares en ambos hipocondrios y en pelvis.
        La ecografía urológica pone de manifiesto la existencia de quistes corticales simples en riñón derecho, vejiga sin alteraciones con buena capacidad y próstata con un peso de 30 g.
        En la UIV se observa normofuncionalismo renal bilateral, calcificaciones sobre silueta renal derecha y uréteres arrosariados con imágenes de adición en el tercio superior de ambos uréteres, en relación a pseudodiverticulosis ureteral. El cistograma demuestra una vejiga con buena capacidad, pero paredes trabeculadas en relación a vejiga de esfuerzo. La TC abdominal es normal.
        La cistoscopia descubre la existencia de pequeñas tumoraciones vesicales, realizándose resección transuretral con el resultado anatomopatológico de carcinoma urotelial superficial de vejiga.
        Remitido por: Ignacio Navarro Cuéllar c/ del Abedul 5-7, 2º dcha 28036 Madrid, España E-mail: nnavcu@hotmail.com.
        
        Ejemplo - Informe en formato .xml: lo que debes generar
        <?xml version='1.0' encoding='UTF-8'?>
        <MEDDOCAN>
          <TEXT>
        Ejemplo - Informe en formato .txt: 
        Datos del paciente.
        Nombre:  <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">Ernesto</TAG>.
        Apellidos: <TAG TYPE="NOMBRE_SUJETO_ASISTENCIA">Rivera Bueno</TAG>.
        NHC: <TAG TYPE="ID_SUJETO_ASISTENCIA">368503</TAG>.
        NASS: <TAG TYPE="ID_ASEGURAMIENTO">26 63514095</TAG>.
        Domicilio:  <TAG TYPE="CALLE">Calle Miguel Benitez 90</TAG>.
        Localidad/ Provincia: <TAG TYPE="TERRITORIO">Madrid</TAG>.
        CP: <TAG TYPE="TERRITORIO">28016</TAG>.
        Datos asistenciales.
        Fecha de nacimiento: <TAG TYPE="FECHAS">03/03/1946</TAG>.
        País: <TAG TYPE="PAIS">España</TAG>.
        Edad: <TAG TYPE="EDAD_SUJETO_ASISTENCIA">70 años</TAG> Sexo: <TAG TYPE="SEXO_SUJETO_ASISTENCIA">H</TAG>.
        Fecha de Ingreso: <TAG TYPE="FECHAS">12/12/2016</TAG>.
        Médico:  <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Ignacio</TAG> <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Navarro Cuéllar</TAG> NºCol: <TAG TYPE="ID_TITULACION_PERSONAL_SANITARIO">28 28 70973</TAG>.
        Informe clínico del paciente: Paciente de <TAG TYPE="EDAD_SUJETO_ASISTENCIA">70 años</TAG> de edad, minero jubilado, sin alergias medicamentosas conocidas, que presenta como antecedentes personales: accidente laboral antiguo con fracturas vertebrales y costales; intervenido de enfermedad de Dupuytren en mano derecha y by-pass iliofemoral izquierdo; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; enolismo activo, fumador de 20 cigarrillos / día.
        Es derivado desde Atención Primaria por presentar hematuria macroscópica postmiccional en una ocasión y microhematuria persistente posteriormente, con micciones normales.
        En la exploración física presenta un buen estado general, con abdomen y genitales normales; tacto rectal compatible con adenoma de próstata grado I/IV.
        En la analítica de orina destaca la existencia de 4 hematíes/ campo y 0-5 leucocitos/campo; resto de sedimento normal.
        Hemograma normal; en la bioquímica destaca una glucemia de 169 mg/dl y triglicéridos de 456 mg/dl; función hepática y renal normal. PSA de 1.16 ng/ml.
        Las citologías de orina son repetidamente sospechosas de malignidad.
        En la placa simple de abdomen se valoran cambios degenerativos en columna lumbar y calcificaciones vasculares en ambos hipocondrios y en pelvis.
        La ecografía urológica pone de manifiesto la existencia de quistes corticales simples en riñón derecho, vejiga sin alteraciones con buena capacidad y próstata con un peso de 30 g.
        En la UIV se observa normofuncionalismo renal bilateral, calcificaciones sobre silueta renal derecha y uréteres arrosariados con imágenes de adición en el tercio superior de ambos uréteres, en relación a pseudodiverticulosis ureteral. El cistograma demuestra una vejiga con buena capacidad, pero paredes trabeculadas en relación a vejiga de esfuerzo. La TC abdominal es normal.
        La cistoscopia descubre la existencia de pequeñas tumoraciones vesicales, realizándose resección transuretral con el resultado anatomopatológico de carcinoma urotelial superficial de vejiga.
        Remitido por: <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Ignacio</TAG> <TAG TYPE="NOMBRE_PERSONAL_SANITARIO">Navarro Cuéllar</TAG> <TAG TYPE="CALLE">c/ del Abedul 5-7, 2º dcha</TAG> <TAG TYPE="TERRITORIO" >28036</TAG> <TAG TYPE="TERRITORIO" >Madrid</TAG>, <TAG TYPE="PAIS">España</TAG> E-mail: <TAG TYPE="CORREO_ELECTRONICO">nnavcu@hotmail.com</TAG>.
        </TEXT>
        </MEDDOCAN>
        
        Recordá que en ningún caso debes incluir advertencias, explicaciones ni descripciones sobre la tarea, sobre la instrucción que te he dado o sobre cuestiones de funcionamiento del modelo de lenguaje.
        """},
     ]

# Configuración de tokens
MAX_CONTEXT_TOKENS = 8192
MAX_GENERATION_TOKENS = 4000
MAX_INPUT_TOKENS = MAX_CONTEXT_TOKENS - MAX_GENERATION_TOKENS

# Procesar cada archivo .txt
for filename in os.listdir(input_dir):
    if filename.endswith(".txt"):
        filepath = os.path.join(input_dir, filename)
        with open(filepath, "r", encoding="utf-8") as f:
            texto = f.read()

        # Crear mensaje estilo chat
        prompt_text = prompt[0]["content"]
        messages = [
            {"role": "system", "content": prompt_text},
            {"role": "user", "content": texto}
        ]

        # Calcular tokens de entrada
        full_prompt = prompt_text + texto
        total_tokens = len(tokenizer.encode(full_prompt))
        print(f"{filename}: Tokens de entrada: {total_tokens}")

        # Truncar el prompt si se pasa del límite permitido
        if total_tokens > MAX_INPUT_TOKENS:
            print(f"Truncando prompt: {filename}")
            # Calcular los tokens disponibles para el prompt
            max_tokens_prompt = MAX_INPUT_TOKENS - len(tokenizer.encode(texto))
            
            # Truncar el prompt para ajustarlo al límite de tokens
            prompt_tokens = tokenizer.encode(prompt[0]["content"])
            truncated_prompt_tokens = prompt_tokens[:max_tokens_prompt]
            
            # Decodificar los tokens truncados y actualizar el prompt
            truncated_prompt = tokenizer.decode(truncated_prompt_tokens, skip_special_tokens=True)
            messages[0]["content"] = truncated_prompt

        # Generar texto
        output = pipeline(messages, max_new_tokens=MAX_GENERATION_TOKENS)

        # Extraer solo el contenido generado por el modelo
        respuesta = output[0]["generated_text"][2]["content"]

        # Guardar en .xml
        output_filename = os.path.splitext(filename)[0] + ".xml"
        output_path = os.path.join(output_dir, output_filename)
        with open(output_path, "w", encoding="utf-8") as out_f:
            out_f.write(respuesta)

        print(f"Procesado: {filename} → {output_filename}")

print("Proceso completado.")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


390535044.txt: Tokens de entrada: 3410


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 390535044.txt → 390535044.xml
990720341.txt: Tokens de entrada: 3324


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Procesado: 827278311.txt → 827278311.xml
262107453.txt: Tokens de entrada: 3276


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 262107453.txt → 262107453.xml
262291314.txt: Tokens de entrada: 3155


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Procesado: 262291314.txt → 262291314.xml
743560436.txt: Tokens de entrada: 2754
Procesado: 743560436.txt → 743560436.xml
Proceso completado.
